# Multitask Model
This code is complete end to end building of a multitask model. Different steps in the code are : 
1. Image preprocessing 
2. Model design
3. Model Train
4. Post training analysis

## Requirements 
| Library | version | version name |
| :---        |    :----:   |   ------:  |
| cudatoolkit |   9.0 | h13b8566_0 |
| cudnn |                     7.6.5 |                cuda9.0_0 |  
|ipykernel|                 5.3.4|            py37h5ca1d4c_0|    
|ipython |                  7.18.1|           py37h5ca1d4c_0|    
|jupyter_client|            6.1.7|                      py_0|    
|jupyter_core|              4.6.3|                    py37_0|    
|keras-applications|        1.0.8|                      py_1|  
|keras-preprocessing|       1.1.0|                      py_1 | 
|matplotlib|                3.3.3|                    pypi_0|    
|matplotlib-base|           3.3.2|            py37h817c723_0|  
|nibabel|                   3.2.1|                    pypi_0|    
|numpy|                     1.19.2|           py37h54aff64_0|
|opencv|                    3.4.2|            py37h6fd60c2_1|  
|pandas|                    1.1.3|            py37he6710b0_0|  
|pillow|                    8.0.1|            py37he98fc37_0|  
|py-xgboost|                0.90|             py37he6710b0_1|    
|python|                    3.7.9|                h7579374_0|  
|scikit-image     |         0.17.2|                   pypi_0|    
|scikit-learn     |         0.23.2|           py37h0573a6f_0|    
|scipy            |         1.5.2  |          py37h0b6359f_0|  
|seaborn          |         0.11.0 |                    py_0|  
|tensorboard     |          1.14.0 |          py37hf484d3e_0|  
|tensorflow     |           1.14.0 |         gpu_py37hae64822_0|  
|tensorflow-gpu|            1.14.0 |              h0d30ee6_0|  

In [ ]:
# Importing the required libraries 

import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow import keras

from sklearn.preprocessing import LabelEncoder


from tensorflow.keras import backend as K
import nibabel as nib
import cv2
import time
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from tensorflow.keras.utils import to_categorical

In [ ]:
from prep_data import get_roi, get_all_subjects, get_subject, get_subject_list, pad_to_shape, remove_padding
from PatchGenerator import PatchGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping, TensorBoard
from tensorflow.keras.applications.resnet50 import preprocess_input
import models

## Data Preprocessing
1. Importing CSV with labels and file paths
2. Label Encodeing 
2. Train-Test split
3. Image processing
4. Saving images as numpy arrays

### Importing CSV
The csv file is generated in the data prep notebook. The file consists of all the requiered labels and patient ID. They also consist the path to the image files of all modalities.

In [ ]:
# Csv import : Csv contains file paths to all modality images and 4 labels 
df = pd.read_csv('../Data/Final_trainData_Multiclass.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df = df[df['MGMT promoter status'].isna() == False]

In [ ]:
gb = df.groupby(['Grade'])
gb['MGMT promoter status'].value_counts()

In [ ]:
gb = df.groupby(['Grade'])
gb['IDH status'].value_counts()

In [ ]:
gb = df.groupby(['Grade'])
gb['IDH-1P19Q Subtype'].value_counts()

### LabelEncoding 
- The labels are encoded as integers. 
- LabelEncoder() from sklearn is used.
- Encoded labels are added to the dataframe as another columns


In [ ]:
le = LabelEncoder()
df['Grade_Le'] = le.fit_transform(df['Grade'])
df['IDH_1P19Q_Le'] = le.fit_transform(df['IDH-1P19Q Subtype'])
df['MGMT_Le'] = le.fit_transform(df['MGMT promoter status'])
df.head()

In [ ]:
df.set_index('Patient ID', inplace = True)

In [ ]:
#Label is arranged as Grade, IDHstatus, MGMT, 1P19Q

Labels = np.c_[df['Grade_Le'], df['Class'], df['MGMT_Le'], df['IDH_1P19Q_Le']]

In [ ]:
# df_train, df_test, Labels_train, Labels_test = train_test_split(df,Labels,test_size=0.20,random_state=9,stratify=Labels)
df_train = df[df['Train_Split']=='train']
Labels_train = Labels[df['Train_Split']=='train']
df_test = df[df['Train_Split']=='test']
Labels_test = Labels[df['Train_Split']=='test']

In [ ]:
print(Labels)

In [ ]:
print('Train:')
print(Labels_train.value_counts())

print('Test:')
print(Labels_test.value_counts())

"""
G3_0 = 0
G3_1 = 1
G4_0 = 2
G4_1 = 3
"""


In [ ]:
tmp = df_test.iloc[0]['FLAIR']
img = nib.load(tmp).get_fdata()
print(img.shape)

In [ ]:
plt.imshow(img[:,:,80],cmap='gray')

In [ ]:
Labels_test = np.array(Labels_test)
Labels_train = np.array(Labels_train)

### Image preprocessing

Code cells below will now process our split data and give us numpy arrays.
The following preprocessing is done on the data:
1. Data is converted to numpy array
2. Slices are taken over the 3D image
3. z2 normalization is performed over each slice
4. A bounding box is taken over each slice for the area around the mask
5. The masked box is then taken over the slice and only the tumor part is considered
6. The bounded slices are then stacked over each other also labels are also stacked in the same order as that of the slices.
7. Final numpy arrays are then returned and saved for further use in model training

In [ ]:
FLAIR_test_data_list, test_mask_list = get_all_subjects(df_test,'FLAIR','Mask',label_values=[1,2,4],transpose_axes=[2,0,1],norm_type='zscore')
T1ce_test_data_list, _ = get_all_subjects(df_test,'T1ce','Mask',label_values=[1,2,4],transpose_axes=[2,0,1],norm_type='zscore')
T2_test_data_list, _ = get_all_subjects(df_test,'T2','Mask',label_values=[1,2,4],transpose_axes=[2,0,1],norm_type='zscore')

In [ ]:
start = time.time()

number_of_slices = 3 ## number of slices to load from each patient

X_test_orig = []
Y_test_orig = []
mask_test_orig = []
modality_list = []

for idx, msk in enumerate(test_mask_list):
    
    FLAIR_img = FLAIR_test_data_list[idx]
    T1ce_img = T1ce_test_data_list[idx]
    T2_img = T2_test_data_list[idx]    
    label = Labels_test[idx]
    
    print('Class: ',label,'\tName : ',df_test.index[idx])
    hMin, hMax, wMin, wMax, dMin, dMax = get_roi(msk,10)    
    
    ## if you want all slices
    imp_slices = list(np.arange(hMin,hMax))
    
    ## important Axial slices
    roi_areas = [(slc,area) for slc,area in enumerate(np.sum(msk,axis=(1,2)))]
    roi_areas = sorted(roi_areas,key=lambda x: x[1],reverse=True)
    imp_slices = [x for x,_ in roi_areas[0:number_of_slices]]        
    
    for slc in imp_slices:
        
        modalities = ['FLAIR','T1ce','T2']
        modality_mapping = {'FLAIR':1,'T1ce':2,'T2':3}
        data_dict = {'FLAIR':FLAIR_img,'T1ce':T1ce_img,'T2':T2_img}
        
        for mod in modalities:
                        
            tmp_slice = data_dict[mod][slc]
            tmp_slice = data_dict[mod][slc][wMin:wMax,dMin:dMax]
            tmp_slice = resize(tmp_slice,[128,128],anti_aliasing=True)                
            
            modality_list.append(modality_mapping[mod])
            Y_test_orig.append(label)
            X_test_orig.append(tmp_slice)        
        
end = time.time()
print(end-start)

X_test_orig = np.stack(X_test_orig, axis=0)
X_test_orig = np.expand_dims(X_test_orig, -1)

Y_test_orig = np.stack(Y_test_orig,axis=0)
print('testing : ',np.unique(Y_test_orig,return_counts=True))
Y_test = to_categorical(Y_test_orig,num_classes=None)

print(X_test_orig.shape,Y_test.shape)

In [ ]:
print(Y_test_orig.shape)

In [ ]:
print(Y_test)

In [ ]:
del(FLAIR_test_data_list) # save memory
del(T2_test_data_list) # save memory
del(T1ce_test_data_list) # save memory
del(test_mask_list) # save memory

In [ ]:
FLAIR_train_data_list, train_mask_list = get_all_subjects(df_train,'FLAIR','Mask',label_values=[1,2,4],transpose_axes=[2,0,1],norm_type='zscore')
T1ce_train_data_list, _ = get_all_subjects(df_train,'T1ce','Mask',label_values=[1,2,4],transpose_axes=[2,0,1],norm_type='zscore')
T2_train_data_list, _ = get_all_subjects(df_train,'T2','Mask',label_values=[1,2,4],transpose_axes=[2,0,1],norm_type='zscore')

In [ ]:
start = time.time()

number_of_slices = 20

X_train_orig = []
Y_train_orig = []
modality_list = []

for idx, msk in enumerate(train_mask_list):
    
#     FLAIR_img = FLAIR_train_data_list[idx]
#     T1ce_img = T1ce_train_data_list[idx]
#     T2_img = T2_train_data_list[idx]    
    label = Labels_train[idx]
    
    print('Class: ',label,'\tName : ',df_train.index[idx])
    hMin, hMax, wMin, wMax, dMin, dMax = get_roi(msk,10)    
    
    ## important Axial slices
    roi_areas = [(slc,area) for slc,area in enumerate(np.sum(msk,axis=(1,2)))]
    roi_areas = sorted(roi_areas,key=lambda x: x[1],reverse=True)
    imp_slices = [x for x,_ in roi_areas[0:number_of_slices]]
    
    for slc in imp_slices:
        
        modalities = ['FLAIR','T1ce','T2']
        modality_mapping = {'FLAIR':1,'T1ce':2,'T2':3}
        data_dict = {'FLAIR':FLAIR_img,'T1ce':T1ce_img,'T2':T2_img}
        
        for mod in modalities:
                        
            tmp_slice = data_dict[mod][slc]
            tmp_slice = data_dict[mod][slc][wMin:wMax,dMin:dMax]
            tmp_slice = resize(tmp_slice,[128,128],anti_aliasing=True)                
            
            modality_list.append(modality_mapping[mod])
            Y_train_orig.append(label)
            X_train_orig.append(tmp_slice)        
        
end = time.time()
print(end-start)

X_train_orig = np.stack(X_train_orig, axis=0)
X_train_orig = np.expand_dims(X_train_orig, -1)

Y_train_orig = np.stack(Y_train_orig,axis=0)
print('training : ',np.unique(Y_train_orig,return_counts=True))
Y_train = to_categorical(Y_train_orig,num_classes=None)

print(X_train_orig.shape,Y_train.shape)

In [ ]:
del(FLAIR_train_data_list) # save memory
del(T2_train_data_list) # save memory
del(T1ce_train_data_list) # save memory
del(train_mask_list) # save memory

In [ ]:
x_train = X_train_orig
x_test = X_test_orig
y_train = Y_train_orig
y_test = Y_test_orig

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

In [ ]:
# Saving single modality images as numpy array

np.save('../Data/X_train_ML_3.npy',x_train)
np.save('../Data/Y_train_ML_3.npy',y_train)
np.save('../Data/X_test_ML_3.npy',x_test)
np.save('../Data/Y_test_ML_3.npy',y_test)


In [ ]:
# Splitting modalities and assignning to individual arrays
x_train_FLAIR = x_train[np.arange(0,4500,3),:,:,0]
x_train_T1ce = x_train[np.arange(1,4500,3),:,:,0]
x_train_T2 = x_train[np.arange(2,4500,3),:,:,0]

x_test_FLAIR = x_test[np.arange(0,171,3),:,:,0]
x_test_T1ce = x_test[np.arange(1,171,3),:,:,0]
x_test_T2 = x_test[np.arange(2,171,3),:,:,0]

print(x_train_FLAIR.shape,x_train_T1ce.shape, x_train_T2.shape)
print(x_test_FLAIR.shape,x_test_T1ce.shape,x_test_T2.shape)

In [ ]:
# Stacking the 3 modalites on each other in the order FLAIR, T1ce, T2
x_train = np.stack([x_train_FLAIR,x_train_T1ce, x_train_T2],axis=-1)
x_test = np.stack([x_test_FLAIR,x_test_T1ce, x_test_T2],axis=-1)
print(x_train.shape,x_test.shape)

In [ ]:
# Arranging labels same as images
y_train = y_train[np.arange(0,len(y_train),3)]
y_test = y_test[np.arange(0,len(y_test),3)]
print(y_train.shape,y_test.shape)

In [ ]:
# Saving the stacked models
np.save('../Data/X_train_ML_3mod.npy',x_train)
np.save('../Data/Y_train_ML_3mod.npy',y_train)
np.save('../Data/X_test_ML_3mod.npy',x_test)
np.save('../Data/Y_test_ML_3mod.npy',y_test)


## Datagen

In [ ]:
x_train = np.load('../Data/X_train_ML_2.npy')
y_train = np.load('../Data/Y_train_ML_2.npy')
x_test = np.load('../Data/X_test_ML_2.npy')
y_test = np.load('../Data/Y_test_ML_2.npy')



print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

In [ ]:
plt.imshow(x_test[10][:,:,0], cmap = 'gray')

In [ ]:
print(y_test)

In [ ]:
bkp_X_train = np.copy(x_train)
bkp_Y_train = np.copy(y_train)
bkp_X_test = np.copy(x_test)
bkp_Y_test = np.copy(y_test)

In [ ]:
"""def grayscale_to_3channel(x):
    x = np.squeeze(x)
    return np.stack([x,x,x],axis=-1)

X_train = grayscale_to_3channel(bkp_X_train)
X_test = grayscale_to_3channel(bkp_X_test)

input_shape = X_train.shape[1:]
print(input_shape)

# X_train = preprocess_input(X_train)
# X_test = preprocess_input(X_test)"""

In [ ]:
del(bkp_X_test)
del(bkp_X_train)
del(bkp_Y_test)
del(bkp_Y_train)

In [ ]:
datagen = ImageDataGenerator(
    width_shift_range = [0.2, 0.3],
    height_shift_range = [0.2, 0.3],    
    vertical_flip = True,
    horizontal_flip = True,
    rotation_range=15,
    zoom_range=[0.5,1.0],
    channel_shift_range=0.2
#     fill_mode = 'constant',
#     cval=0,
)

from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

y_integers = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
train_sample_weights = compute_sample_weight('balanced',y_train)
train_class_weights = dict(enumerate(class_weights))

print(train_class_weights)

y_integers = np.argmax(y_test, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
test_sample_weights = compute_sample_weight('balanced',y_test)
test_class_weights = dict(enumerate(class_weights))

print(test_class_weights)

In [ ]:
y_test[:,1]

In [ ]:
y_test[:,0]

In [ ]:
# Custom generator function to map the 4 labels to the inputs
# This is necessary as we have 4 outputs for our model due to 4 tasks

def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(4)])

In [ ]:
batch_size = 64

train_datagen = datagen.flow(x_train,y_train,
                             batch_size=batch_size,
                             shuffle=True,                             
                            )


test_datagen = ImageDataGenerator().flow(x_test,y_test,batch_size=batch_size,
                                         shuffle=True)

#test_datagen = datagen.flow(X_test,y_test,batch_size=batch_size,
#                                         shuffle=True)

print(len(train_datagen),len(test_datagen))

In [ ]:
batch = test_datagen.next()
subject = np.random.randint(0,len(batch[1]))
print(subject,batch[1][subject])

plt.figure()
#plt.subplot(1,2,1)
plt.imshow(batch[0][subject,:,:,0],cmap='gray')

print(np.mean(batch[0])0.,np.std(batch[0]))

In [ ]:
test_datagen.reset()

## Model Design
1. fire_module from squeeze net architeture (paper : https://arxiv.org/abs/1602.07360)
2. Multitask model design with all the branches.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping, TensorBoard
# import models
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet50,DenseNet169,InceptionResNetV2,VGG16
from tensorflow.keras.layers import Flatten,concatenate,Input,Activation, GlobalAveragePooling2D,GlobalMaxPooling2D, Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Lambda,AveragePooling2D

In [ ]:
# Squeezenet fireblock module design

sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Conv2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Conv2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Conv2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

In [ ]:
def res50_model(ip):
    
    # Importing the resnet50 architecture with imagenet weights from the keras applications 

    K_res50 = keras.applications.ResNet50(input_tensor = ip, include_top=False, weights='imagenet', input_shape=[128, 128, 3])
  
    K_res50_l = K_res50.layers[-10]
    x1 = K_res50.output # Taking the output skipping the last resnet block
    x = K_res50_l.output # Taking the output from the resnet module
    
    # Deciding the trainablity of the layers
    for layer in K_res50.layers[12:50]:
        layer.trainable = False 
    
    x = MaxPool2D(pool_size=(2, 2), strides=None, padding='same', name='CMaxPool')(x)
    
    #Grade Prediction layers
    
    output1 = fire_module(x, fire_id='op1_0', squeeze=64, expand=256)
    output1 = fire_module(output1, fire_id='op1_1', squeeze=64, expand=128)
    output1 = keras.layers.BatchNormalization(name='BatchNorm_op1_0')(output1)
    output1 = keras.layers.GlobalAveragePooling2D(name='gap_op1')(output1)
    #output1 = keras.layers.BatchNormalization(name='BatchNorm_op1_1')(output1)
    output1 = keras.layers.Dropout(0.3,name='Dropout_output1')(output1)
    #output1 = keras.layers.Dense(8, activation='sigmoid', name='output1_Dense1')(output1)
    output1 = keras.layers.Dense(1, activation='sigmoid', name='output1')(output1)
    
    #IDH status prediction layers
    
    #output2 = Conv2D(256, (1, 1), padding='valid', name = 'conv_op2')(x1)
    #output2 = keras.layers.BatchNormalization(name='BatchNorm_op2_0')(output2)
    output2 = fire_module(x1, fire_id='op2_0', squeeze=64, expand=256)
    #output2 = keras.layers.BatchNormalization(name='BatchNorm_op2_1')(output2)
    #output2 = fire_module(output2, fire_id='op2_1', squeeze=4, expand=8)
    output2 = keras.layers.GlobalMaxPooling2D(name='gmp_op2')(output2)
    output2 = keras.layers.BatchNormalization(name='BatchNorm_op2')(output2)
    output2 = keras.layers.Dropout(0.3,name='Dropout_output2')(output2)
    output2 = keras.layers.Dense(256, name='output2_Dense2')(output2)
    output2 = keras.layers.LeakyReLU(alpha=0.3)(output2)
    output2 = keras.layers.Dropout(0.4,name='Dropout_1_output2')(output2)
    
    #output2 = keras.layers.Dense(2, activation='sigmoid', name='output2_0')(output2)
    output2 = keras.layers.Dense(1, activation = 'sigmoid', name = 'output2')(output2)
    
    #MGMT prediction layers
    
    output3 = fire_module(x, fire_id='op3', squeeze=64, expand=128)
    output3 = keras.layers.BatchNormalization(name='BatchNorm_op3_0')(output3)
    output3 = fire_module(output3, fire_id='op3_1', squeeze=16, expand=32)
    output3 = keras.layers.GlobalMaxPooling2D(name='gap_op3')(output3)
    output3 = keras.layers.Dropout(0.3,name='Dropout_output3')(output3)
    #output3 = keras.layers.Dense(8, activation='sigmoid', name='output3_Dense2')(output3)
    output3 = keras.layers.Dense(1, activation='sigmoid', name='output3')(output3)
    
    #1P19q prediction layers
    
    output4_left = fire_module(x, fire_id='op4_0', squeeze=64, expand=128)
    output4_left = keras.layers.BatchNormalization(name='BatchNorm_op4_0')(output4_left)
    output4_left = keras.layers.Dropout(0.3,name='Dropout_output4_0')(output4_left)
    output4_left = fire_module(output4_left, fire_id='op4_1', squeeze=32, expand=64)
    output4_left = keras.layers.GlobalAveragePooling2D(name='gap_op4')(output4_left)
    output4_left = keras.layers.Dropout(0.3,name='Dropout_output4_1')(output4_left)
    #output4_left = keras.layers.Dense(8, activation='sigmoid', name='output4_l_Dense1')(output4_left)
    output4_left = keras.layers.Dense(2, activation='sigmoid', name='output4_l_Dense2')(output4_left)
    
    output4_right = keras.layers.Dense(2, activation='sigmoid', name='buffer_op2_op4')(output2)
    
    output4 = concatenate([output4_left, output4_right], axis=-1, name='output4_concat')
    output4 = keras.layers.Dense(1, activation='sigmoid', name='output4')(output4)
    
    return [output1, output2, output3, output4]


In [ ]:
ip1=Input(shape=(128,128,3))
res50=Model(inputs=ip1,outputs=res50_model(ip1))
res50.summary()

In [ ]:
# Visualising the model
from tensorflow.keras.utils import plot_model
plot_model(res50, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
"""
Defining custom loss function
For the missing labels value needs to be set to -1 which is the mask value.
This loss function masks the missing labels and thus no training happens.
"""
mask_value = -1
def masked_loss_function(y_true, y_pred):
    mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
    return K.binary_crossentropy(y_true * mask, y_pred * mask)



In [ ]:
adam = keras.optimizers.Adam(learning_rate=0.000003)

res50.compile(adam,['binary_crossentropy','binary_crossentropy', masked_loss_function, masked_loss_function],metrics=['accuracy'])

In [ ]:
train_steps = len(train_datagen)
test_steps = len(test_datagen)
print(train_steps, test_steps)

In [ ]:
filename=os.path.join('logs','TCGA_res50_MT_3mod.csv')
filepath=os.path.join('weights','TCGA_res50_MT_3mod.hdf5')
csv_log = CSVLogger(filename, separator=',', append=True)
checkpoint = ModelCheckpoint(filepath, monitor='val_output1_acc', verbose=1, save_best_only=True)
rl = ReduceLROnPlateau(monitor='val_output1_acc',patience=5,min_delta=0.001,cooldown=5,factor=0.1)
tb = TensorBoard('./logs',histogram_freq=0)
callbacks_list = [csv_log,
                  checkpoint,
                  rl,
                  tb
                 ]

In [ ]:
if os.path.exists('weights/TCGA_res50_MT_10.hdf5'):
    res50.load_weights(filepath)

In [ ]:
print(filepath)

In [ ]:
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
epochs = 500
res50_H = res50.fit(generator_wrapper(train_datagen), steps_per_epoch=train_steps, epochs=epochs,
                    verbose = 1,validation_data=generator_wrapper(test_datagen),validation_steps=test_steps,
                    callbacks=callbacks_list)

In [ ]:
# create a new figure for the accuracies
accuracyNames = ["output1_acc", "output2_acc", "output3_acc", 'output4_acc']
plt.style.use("ggplot")
(fig, ax) = plt.subplots(4, 1, figsize=(8, 8))
# loop over the accuracy names
for (i, l) in enumerate(accuracyNames):
    # plot the loss for both the training and validation data
    ax[i].set_title("Accuracy for {}".format(l))
    ax[i].set_xlabel("Epoch #")
    ax[i].set_ylabel("Accuracy")
    ax[i].plot(np.arange(0, epochs), res50_H.history[l], label=l)
    ax[i].plot(np.arange(0, epochs), res50_H.history["val_" + l],
        label="val_" + l)
    ax[i].legend()
# save the accuracies figure
plt.tight_layout()
plt.show()


In [ ]:
lossNames = ["output1_loss", "output2_loss", "output3_loss", 'output4_loss']
plt.style.use("ggplot")
(fig, ax) = plt.subplots(4, 1, figsize=(8, 8))
# loop over the loss names
for (i, l) in enumerate(lossNames):
    # plot the loss for both the training and validation data
    ax[i].set_title("loss for {}".format(l))
    ax[i].set_xlabel("Epoch #")
    ax[i].set_ylabel("loss")
    ax[i].plot(np.arange(0, epochs), res50_H.history[l], label=l)
    ax[i].plot(np.arange(0, epochs), res50_H.history["val_" + l],
        label="val_" + l)
    ax[i].legend()
# save the accuracies figure
plt.tight_layout()
plt.show()

In [ ]:
#saving the model with weights
res50.save('../Model/TCGA_Multitask_res50_4pred_2.h5')

res50.save_weights('weights/TCGA_Multitask_res50_4pred_2.hdf5')

## Post training analysis

In [ ]:
_test_data, _test_labels = test_datagen.__getitem__(np.random.randint(0,len(test_datagen)))

In [ ]:
## evaluate the model and predict on testing data
print('Evaluate')
a=res50.evaluate(generator_wrapper(test
                                   _datagen),batch_size=batch_size,verbose=1,steps = 1)
print('val_loss , val_acc: ', a)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (accuracy_score, classification_report,
                              confusion_matrix, roc_auc_score, roc_curve)

def get_roc(y_true, y_pred, positive_class_index=0):
    y_pred = np.copy(y_pred)[:, positive_class_index]
    return {'auroc': roc_auc_score(y_true, y_pred), 'roc': roc_curve(y_true, y_pred)}

In [ ]:
y_pred_proba = res50.predict_generator(test_datagen,verbose=1) 
y_true = y_test
y_pred = y_pred_proba

In [ ]:
len(y_pred_proba[1])

In [ ]:
y_pred = np.array(y_pred).reshape(-1)

In [ ]:
y_test.shape

In [ ]:
y_pred = np.array(y_pred).reshape(152,4)

In [ ]:
print(y_pred)

In [ ]:
print(classification_report(y_true[:,0],y_pred[:,0].round())) # classification report for Grade

In [ ]:
print(classification_report(y_true[:,1],y_pred[:,1].round())) # classification report for IDH

In [ ]:
print(classification_report(y_true[:,2],y_pred[:,2].round())) # classification report for MGMT

In [ ]:
print(classification_report(y_true[:,3],y_pred[:,3].round())) # classification report for 1p19q

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
n_classes = 4
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true[:,i],y_pred[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    #plt.plot([0, 1], [0, 1], 'k--')
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()